<a href="https://colab.research.google.com/github/Jaimemorillo/ShouldIwatchThisMovie/blob/master/translate_overviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# Drop duplicates

In [3]:
import pandas as pd
import numpy as np
import time

import re

from tqdm.notebook import tqdm
tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
data = pd.read_csv("gdrive/My Drive/TFG/overview_translate_3.csv", sep='~', encoding='utf-8', lineterminator='\n')

In [5]:
!pip install translators

In [6]:
import translators as ts

API = 'google'

def translator_constructor(api):
    if api == 'google':
        return ts.google
    elif api == 'bing':
        return ts.bing
    elif api == 'baidu':
        return ts.baidu
    elif api == 'sogou':
        return ts.sogou
    elif api == 'youdao':
        return ts.youdao
    elif api == 'tencent':
        return ts.tencent
    elif api == 'alibaba':
        return ts.alibaba
    elif api == 'yandex':
        return ts.yandex
    else:
        raise NotImplementedError(f'{api} translator is not realised!')

translator = translator_constructor(API)

def translate(x, lang='en'):
    try:
        return translator(x, 'es', lang)
    except:
        print('KO')
        time.sleep(60)
        return translate(x, lang='en')

Using United States server backend.


In [7]:
translator = translator_constructor('google')
data.loc[901:, 'first_sentence_en'] = data.loc[901:, 'first_sentence'].progress_apply(lambda x: translate(x))

In [9]:
data.to_csv("gdrive/My Drive/TFG/overview_translate_4.csv", sep='~', index=False, encoding='utf-8')

In [13]:
data = data.drop_duplicates(subset=['id'])

In [16]:
data.to_csv("gdrive/My Drive/TFG/overview_translate_final.csv", sep='~', index=False, encoding='utf-8')

In [23]:
data['first_sentence_en'] = data['first_sentence_en'].progress_apply(lambda x: " ".join(x.split(" ")[0:15]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [25]:
data[['first_sentence_en']].to_csv("gdrive/My Drive/TFG/overview_translate_text_gen.csv", sep=',', index=False, encoding='utf-8')

In [31]:
data.loc[0, 'first_sentence_en'][0:200]

'Princess Leia leader of rebellious movement who wishes to reinstate the Republic in the galaxy'